In [3]:
import sys
import time
import argparse
import numpy as np
import pandas as pd
import h5py
from datetime import datetime
from tsforest import forecaster
from tsforest.metrics import compute_rmse, compute_rmsle
from scripts.utils import reduce_mem_usage
from scripts.config import get_model_params
from scripts.scaling import target_transform, target_inverse_transform

In [5]:
model_class_name = "LightGBMForecaster"
model_class = getattr(forecaster, model_class_name)

In [6]:
train_data = pd.read_hdf('data/train_data_nw.h5', 'train_data')
train_data.rename({"timestamp":"ds", "meter_reading":"y"}, axis=1, inplace=True)

In [7]:
robust_scaler = pd.read_csv("data/robust_scaler.csv")
train_data = target_transform(train_data, robust_scaler, target="y")

In [9]:
h5f = h5py.File("data/valid_sm_custom_4fold.h5", "r")
valid_indexes = [h5f[key][:] for key in h5f.keys()]

In [10]:
model_kwargs = {"model_params":get_model_params(model_class_name),
                "feature_sets":['calendar', 'calendar_cyclical'],
                "exclude_features":["year","days_in_month"],
                "categorical_features":{"building_id":"default",
                                        "meter":"default",
                                        "site_id":"default",
                                        "primary_use":"default"},
                "ts_uid_columns":["building_id","meter"],
                "detrend":False,
                "target_scaler":None}

In [12]:
n_folds = len(valid_indexes)

for i,valid_index in enumerate(valid_indexes):
    fcaster = model_class(**model_kwargs)
    fcaster.prepare_features(train_data=train_data, valid_index=valid_index)
    train_features = reduce_mem_usage(fcaster.train_features)
    valid_features = reduce_mem_usage(fcaster.valid_features)
    
    train_features.to_hdf(f"data/train_features_nw_{n_folds}fold.h5", key=f"fold{i}")
    valid_features.to_hdf(f"data/valid_features_nw_{n_folds}fold.h5", key=f"fold{i}")

Mem. usage decreased to 909.33 Mb (68.0% reduction)
Mem. usage decreased to 303.20 Mb (68.0% reduction)
Mem. usage decreased to 909.56 Mb (68.0% reduction)
Mem. usage decreased to 302.98 Mb (68.0% reduction)
Mem. usage decreased to 909.46 Mb (68.0% reduction)
Mem. usage decreased to 303.07 Mb (68.0% reduction)
Mem. usage decreased to 909.25 Mb (68.0% reduction)
Mem. usage decreased to 303.28 Mb (68.0% reduction)


***